In [ ]:
import os, sys
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd

import ROOT
import root_numpy as rn
from larocv import larocv

rse    = ['run','subrun','event']
rsev   = ['run','subrun','event','vtxid']
rserv  = ['run','subrun','event','roid','vtxid']

In [ ]:
name='bnbdata_5e19_p00'
LLCUT=-12
if 'cosmic' not in name:
    all_df = pd.read_pickle(os.path.join("..","ll_bin","{}_post_LL.pkl".format(name)))
else:
    all_df = pd.read_pickle(os.path.join("..","ll_bin","{}_post_nue.pkl".format(name)))

if name == 'cosmic' or name == 'cosmic2':
    SS = "LL>@LLCUT"
    SS = "angle3d>1.05 & angle3d<1.09"
    print "GOT",all_df.query(SS).index.size,"@ LL "
    pass_df = all_df.query(SS)
    FILE="/data/vgenty/extbnb/out_week082817/vertex/analysis/rse/comb_ana.root"
    
if '1e1p' in name:
    SS = "LL>@LLCUT & scedr<5 & selected1L1P==1 & energyInit<800 & energyInit>200"
    print "GOT",all_df.query(SS).index.size,"@ LL "
    pass_df = all_df.query(SS)
    FILE="/data/vgenty/1e1p/out_week080717/vertex/analysis/rse/comb_ana.root"
    FIN="/data/vgenty/1e1p/out_week080717/ssnet_mcc8"
    FOUT="/data/vgenty/tufts/vertex/vertex_09152017_p00/1e1p/out/"
    
if 'ncpizero' in name:
    print "GOT",all_df.query("LL>@LLCUT & scedr<5").index.size,"@ LL "
    pass_df = all_df.query("LL>@LLCUT & scedr<5")
    FILE="/data/vgenty/ncpizero/out_week080717/vertex/analysis/rse/comb_ana.root"
    FIN  = "/data/vgenty/ncpizero/out_week080717/ssnet_mcc8"
    FOUT = "/data/vgenty/tufts/vertex/vertex_09152017_p00/ncpizero/out/"
    
if name == 'numu':
    print "GOT",all_df.query("LL>@LLCUT & scedr<5").index.size,"@ LL "
    pass_df = all_df.query("LL>@LLCUT & scedr<5")
    FILE="/data/vgenty/1mu1p/out_week080717/vertex/analysis/rse/comb_ana.root"
   
if name == 'inclusive_muon':
    print "GOT",all_df.query("LL>@LLCUT & scedr<5").index.size,"@ LL "
    pass_df = all_df.query("LL>@LLCUT & scedr<5")
    FILE="/data/vgenty/inclusive_muon/out_week080717/vertex/analysis/rse/comb_ana.root"
       
if 'bnb' in name:
    print "GOT",all_df.query("LL>@LLCUT").index.size,"@ LL "
    pass_df = all_df.query("LL>@LLCUT")
    FILE="/data/vgenty/bnb_data/rse/comb_ana.root"
    
rse_df = pd.DataFrame(rn.root2array(FILE,treename='RSEFilter'))
print "GOT",len(rse_df.groupby('fname')),"unique files"

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [ ]:
#
# Generate dump_img bash script
#
name='bnb2'
fout = open("these_{}_pass.sh".format(name),"w+")
ix=-1
for run,subrun,event in pass_df[['run','subrun','event']].values:
    ix+=1
    print "@ix=",ix
    trse = (run,subrun,event)
    fout.write("python dump_img.py")
    fout.write(" ")
    f=os.path.basename(rse_df.set_index(['run','subrun','event']).loc[trse].fname)
    fout.write(os.path.join("{}_in".format(name),f))
    fout.write(" ")
    num = int(f.split(".")[0].split("_")[-1])
    
    PREFIX="vertexout_larcv_"
    #PREFIX="out_"
        
    fout.write(os.path.join("{}_out".format(name),"{}{}.root".format(PREFIX,num)))
    fout.write(" ")
    erv = pass_df.set_index(['run','subrun','event']).loc[trse][['entry','cvtxid']].values
    print erv
    fout.write("%d %d %s &" %(erv[0],erv[1],name))
    fout.write("\n")
    print trse,"\t",f,"%d %d %s &" %(erv[0],erv[1],name)
    print
    
fout.close()

In [ ]:
#
# Generate reana script
#
fout = open("reana_{}.sh".format(name),"w+")
ix=-1
for run,subrun,event in pass_df[['run','subrun','event']].values:
    ix+=1
    print "@ix=",ix
    trse = (run,subrun,event)
    erv = pass_df.set_index(['run','subrun','event']).loc[trse][['entry','cvtxid']].values
    fid = os.path.basename(rse_df.set_index(['run','subrun','event']).loc[trse].fname)
    fid = fid.split(".")[0].split("_")[-1]

    fout.write("python ../../cfg/mac/run_reana.py prod_fullchain_reana.cfg")
    fout.write(" ")
    fout.write(os.path.join("{}".format(name),"ana_{}_{}_{}".format(int(erv[0]),int(erv[1]),fid)))
    fout.write(" ")
    fout.write(os.path.join("{}".format(name),"out_{}_{}_{}".format(int(erv[0]),int(erv[1]),fid)))
    fout.write(" ")
    fout.write("%d %d" %(erv[0],erv[1]))
    fout.write(" ")
    fout.write(os.path.join(FIN,"ssnetout_larcv_{}.root".format(fid)))
    #PREFIX="out_"
    #if name in ['ncpizero','numu','inclusive_muon']:
    PREFIX="vertexout_larcv_"
    fout.write(" ")
    fout.write(os.path.join(FOUT,"{}{}.root".format(PREFIX,fid)))
    fout.write("\n")
    print trse,"\t","%d %d %s &" %(erv[0],erv[1],fid)
    print
    
fout.close()

